<a href="https://colab.research.google.com/github/proteinovayas13/My_works/blob/main/%D0%92%D0%B5%D0%B1_%D1%81%D0%BA%D1%80%D0%B5%D0%B9%D0%BF%D0%B8%D0%BD%D0%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Веб-скрейпинг

Создаю парсер который может собрать 1000 книг с сайта https://books.toscrape.com

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
BASE_URL = 'https://books.toscrape.com/catalogue/'

In [12]:
def parse_book_details(book_url):
    """Парсит детали одной книги"""
    response = requests.get(book_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Получаю UPC код
    upc = soup.find('table', class_='table').find_all('tr')[0].find_all('td')[0].text
    return upc


def parse_page(url):
    """Парсит одну страницу с книгами"""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    books = []

    for book in soup.find_all('article', class_='product_pod'):
        book_name = book.h3.a['title']
        price = book.find('p', class_='price_color').text[1:]  # Убираю символ валюты
        book_url = BASE_URL + book.h3.a['href']

        book_upc = parse_book_details(book_url)
        books.append({
            'id': book_upc,
            'book_name': book_name,
            'price': price,
            'url': book_url
        })

    return books

def scrape_books(max_books=1000):
    """Собирает книги с сайта"""
    all_books = []
    page = 1

    while len(all_books) < max_books:
        url = f"https://books.toscrape.com/catalogue/page-{page}.html"
        print(f"Парсинг страницы {page}: {url}")

        books = parse_page(url)
        if not books:
            print("Больше нет книг для парсинга")
            break

        all_books.extend(books)
        print(f"Найдено книг на странице: {len(books)}. Всего собрано: {len(all_books)}")

        # Проверяю, есть ли следующая страница
        if len(books) < 20:                              # на 1 странице 20 книг, всего 50 страниц
            print("Это последняя страница")
            break

        page += 1

    return all_books[:max_books]

In [14]:
# Запускаю парсинг
if __name__ == "__main__":
    books_data = scrape_books(max_books=1000)
    print(f"\nСобрано книг: {len(books_data)}")

    if books_data:
        print("\nПервые 5 книг:")
        for i, book in enumerate(books_data[:5], 1):
            print(f"{i}. {book['book_name']} - {book['price']} - UPC: {book['id']} - URL: {book['url']}")


Начинаем парсинг книг...
Парсинг страницы 1: https://books.toscrape.com/catalogue/page-1.html
Найдено книг на странице: 20. Всего собрано: 20
Парсинг страницы 2: https://books.toscrape.com/catalogue/page-2.html
Найдено книг на странице: 20. Всего собрано: 40
Парсинг страницы 3: https://books.toscrape.com/catalogue/page-3.html
Найдено книг на странице: 20. Всего собрано: 60
Парсинг страницы 4: https://books.toscrape.com/catalogue/page-4.html
Найдено книг на странице: 20. Всего собрано: 80
Парсинг страницы 5: https://books.toscrape.com/catalogue/page-5.html
Найдено книг на странице: 20. Всего собрано: 100
Парсинг страницы 6: https://books.toscrape.com/catalogue/page-6.html
Найдено книг на странице: 20. Всего собрано: 120
Парсинг страницы 7: https://books.toscrape.com/catalogue/page-7.html
Найдено книг на странице: 20. Всего собрано: 140
Парсинг страницы 8: https://books.toscrape.com/catalogue/page-8.html
Найдено книг на странице: 20. Всего собрано: 160
Парсинг страницы 9: https://books.t

In [18]:
df = pd.DataFrame(books_data)
df.to_csv('books.csv', index=False)

print('Файл books.csv создан!')

df.head(1000)

Файл books.csv создан!


,id,book_name,price,url
0,a897fe39b1053632,A Light in the Attic,51.77,https://books.toscrape.com/catalogue/a-light-i...
1,90fa61229261140a,Tipping the Velvet,53.74,https://books.toscrape.com/catalogue/tipping-t...
2,6957f44c3847a760,Soumission,50.10,https://books.toscrape.com/catalogue/soumissio...
3,e00eb4fd7b871a48,Sharp Objects,47.82,https://books.toscrape.com/catalogue/sharp-obj...
4,4165285e1663650f,Sapiens: A Brief History of Humankind,54.23,https://books.toscrape.com/catalogue/sapiens-a...
...,...,...,...,...
995,cd2a2a70dd5d176d,Alice in Wonderland (Alice's Adventures in Won...,55.53,https://books.toscrape.com/catalogue/alice-in-...
996,bfd5e1701c862ac3,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,https://books.toscrape.com/catalogue/ajin-demi...
997,19fec36a1dfb4c16,A Spy's Devotion (The Regency Spies of London #1),16.97,https://books.toscrape.com/catalogue/a-spys-de...
998,f684a82adc49f011,1st to Die (Women's Murder Club #1),53.98,https://books.toscrape.com/catalogue/1st-to-di...
